In [ ]:
#!/usr/bin/env python

"""
DESCRIPTION

    Script for scraping process parameters and defect parameters for PCA
    Outfitted for use with 'process_parameters.csv'
"""

from __future__ import division
from scipy import stats
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys, os, glob
import numpy as np
import json

def scrape_data(plate_list=['2'], feat_list=None, resp_var=None, stat_param=None):

    # Retrieve all build features
    all_build_data = np.genfromtxt('process_parameters.csv', delimiter=',',
                                    dtype=None)
    
    # Obtain build features of interest
    if feat_list==None:
        print all_build_data[0,:]
        feats = raw_input('Please enter build features as a list of strings: ')
    else:
        pass 
    
    indexes = {}
    indexes['plate'] = np.argwhere(all_build_data[0,:]=='plate')
    for feat in feat_list:
        indexes[feat] = np.argwhere(all_build_data[0,:]==feat)[0][0]
    
     # Obtain user specified defect parameter for regression
    if resp_var==None:
        print json.loads(open('A13_pore-distribution.json', 'rb').read()).keys()
        resp_var = raw_input('Please enter (void specific) response variable as a string: ')
    else:
        pass    
    if stat_param==None:
        stat_param = raw_input("Please enter statistical metric ('mean', 'median', 'mode'): ")
    else:
        pass

    # Iterate through .json files for defect parameters
    processed_parts = []
    response = []    
    for f in glob.glob('*.json'):
        data = json.loads(open(f, 'rb').read())
        ifile = os.path.basename(f)
        processed_parts.append(os.path.splitext(ifile)[0][:3])
        if stat_param=='mean':
            response.append(np.mean(data[resp_var]['values']))
        elif stat_param=='mode':
            response.append(stats.mode(part[resp_var]['values'],axis=None)[0][0])
        else:
            response.append(np.median(data[resp_var]['values']))
    processed_parts = np.reshape(processed_parts, (len(processed_parts),1))
    response = np.reshape(response, (len(response),1))
       
    # Remove zeros from part names in 'processed_parts'
    abbrev_parts = []
    for j in processed_parts:
        if j[0][1]=='0':
            abbrev_parts.append(j[0][0]+j[0][2])
        else:
            abbrev_parts.append(j[0])
    np.reshape(abbrev_parts,(len(abbrev_parts),1))
    
    # Generate ID-response array
    input_data = []
    ID_response = np.concatenate(abbrev_parts, response), axis=1)
    for sample in all_build_data:
    if any(y==sample[plate_idx] for y in plate_list):
        if (any(x==(sample[col_idx]+sample[row_idx]) for x in abbrev_parts)):
            match = np.argwhere((sample[col_idx]+sample[row_idx])
                                    ==ID_response[:,0])
            sample = np.append(sample, ID_response[match,1:])
            input_data.append(sample)
    input_data = np.vstack(input_data)

    
    

In [ ]:
def pcr(input_data):
    
